# Using chained calls for reasoning structured outputs

The initially released versions (September 2024) of [o1](https://openai.com/index/introducing-openai-o1-preview/) reasoning models have advanced capabilities but do not have [structured outputs](https://platform.openai.com/docs/guides/structured-outputs/examples) support. 

This means that requests with o1 don't have reliable type-safety and rely on the prompt itself to return a useful JSON. 

In this guide, we'll explore two methods to prompt o1 models, specifically `o1-preview`, to return a valid JSON format when using the OpenAI API.

# Prompting

The simplest way to return a JSON response using `o1-preview` is to explicitly prompt it. 

Let's run through an example of:
- Fetching a wikipedia page of companies
- Determining which could benefit the most from AI capabilities
- Returning them in a JSON format, which could then be ingested by other systems

In [11]:
import requests
from openai import OpenAI

client = OpenAI()

def fetch_html(url):
    response = requests.get(url)
    if response.status_code == 200:
        return response.text
    else:
        return None

url = "https://en.wikipedia.org/wiki/List_of_largest_companies_in_the_United_States_by_revenue"
html_content = fetch_html(url)

json_format = """
{
    companies: [
        {
            \"company_name\": \"OpenAI\",
            \"page_link\": \"https://en.wikipedia.org/wiki/OpenAI\",
            \"reason\": \"OpenAI would benefit because they are an AI company...\"
        }
    ]
}
"""

response = client.chat.completions.create(
    model="o1-preview",
    messages=[
        {
            "role": "user", 
            "content": f"""
You are a business analyst designed to understand how AI technology could be used across large corporations.

Read the following html and return which companies would benefit from using AI technology: {html_content}. Rank these propects by opportunity by comparing them and show me the top 3. Return only as a JSON with the following format: {json_format}"
"""
        }
    ]
)

print(response.choices[0].message.content)

{
    "companies": [
        {
            "company_name": "Walmart",
            "page_link": "https://en.wikipedia.org/wiki/Walmart",
            "reason": "As the largest retailer, Walmart can benefit from AI technology by enhancing supply chain management, optimizing inventory levels, improving customer experience through personalized recommendations, and streamlining operations for increased efficiency."
        },
        {
            "company_name": "UnitedHealth Group",
            "page_link": "https://en.wikipedia.org/wiki/UnitedHealth_Group",
            "reason": "UnitedHealth Group can leverage AI for predictive analytics, personalized healthcare plans, improved diagnostics, and automating administrative tasks, leading to better patient outcomes and operational efficiency."
        },
        {
            "company_name": "JPMorgan Chase",
            "page_link": "https://en.wikipedia.org/wiki/JPMorgan_Chase",
            "reason": "JPMorgan Chase can utilize AI for frau

```
{
    "companies": [
        {
            "company_name": "Walmart",
            "page_link": "https://en.wikipedia.org/wiki/Walmart",
            "reason": "Walmart could leverage AI technology to optimize supply chain management, improve inventory forecasting, enhance customer experience through personalized recommendations, and streamline operations across its vast retail network."
        },
        {
            "company_name": "JPMorgan Chase",
            "page_link": "https://en.wikipedia.org/wiki/JPMorgan_Chase",
            "reason": "JPMorgan Chase could benefit from AI by implementing advanced fraud detection systems, automating customer service with chatbots, enhancing risk management, and providing personalized financial advisory services using predictive analytics."
        },
        {
            "company_name": "UnitedHealth Group",
            "page_link": "https://en.wikipedia.org/wiki/UnitedHealth_Group",
            "reason": "UnitedHealth Group could utilize AI to improve patient care through predictive analytics, automate administrative tasks, enhance diagnostic accuracy, and develop personalized treatment plans, thus increasing efficiency and patient satisfaction."
        }
    ]
}
```

Note that the response is already quite good - it returns the JSON with the appropriate responses. However, it runs into the same pitfalls as existing use-cases of prompt-only JSON inference: 
- You must manually process this JSON into your type-safe structure
- Model refusals are not [explicitly returned from the API as a separate structure](https://platform.openai.com/docs/guides/structured-outputs/refusals)

# Structured Outputs

Let's now do this with [structured outputs.](https://platform.openai.com/docs/guides/structured-outputs). To enable this functionality, we'll chain our o1 response, with a 4o response that can effectively handle the data returned from the original o1 response. 

In [9]:
import requests
from pydantic import BaseModel
from openai import OpenAI

client = OpenAI()

def fetch_html(url):
    response = requests.get(url)
    if response.status_code == 200:
        return response.text
    else:
        return None

url = "https://en.wikipedia.org/wiki/List_of_largest_companies_in_the_United_States_by_revenue"
html_content = fetch_html(url)

class CompanyData(BaseModel):
    company_name: str
    page_link: str
    reason: str

class CompaniesData(BaseModel):
    companies: list[CompanyData]

json_format = """
{
    \"company_name\": \"OpenAI\",
    \"page_link\": \"https://en.wikipedia.org/wiki/OpenAI\",
    \"reason\": \"OpenAI would benefit because they are an AI company...\"}"
"""

o1_response = client.chat.completions.create(
    model="o1-preview",
    messages=[
        {
            "role": "user", 
            "content": f"""
You are a business analyst designed to understand how AI technology could be used across large corporations.

Read the following html and return which companies would benefit from using AI technology: {html_content}. Rank these propects by opportunity by comparing them and show me the top 3. Return each company with the following information: {CompanyData.model_fields.keys()}"
"""
        }
    ]
)

o1_response_content = o1_response.choices[0].message.content

response = client.beta.chat.completions.parse(
    model="gpt-4o-2024-08-06",
    messages=[
        {
            "role": "user", 
            "content": f"""
Given the following data, format it with the given response format: {o1_response_content}
"""
        }
    ],
    response_format=CompaniesData,
)

print(response.choices[0].message.parsed)

companies=[CompanyData(company_name='Walmart', page_link='https://en.wikipedia.org/wiki/Walmart', reason='As the largest retailer, Walmart can leverage AI for supply chain optimization, inventory management, personalized customer experiences, and operational efficiency.'), CompanyData(company_name='ExxonMobil', page_link='https://en.wikipedia.org/wiki/ExxonMobil', reason='ExxonMobil can benefit from AI in predictive maintenance, optimizing exploration and drilling, analyzing geological data, and enhancing safety measures.'), CompanyData(company_name='UnitedHealth Group', page_link='https://en.wikipedia.org/wiki/UnitedHealth_Group', reason='UnitedHealth Group can utilize AI to improve patient data analysis, diagnostics, personalize treatment plans, detect fraud, and increase operational efficiency.')]


# Conclusion

The o1 models don't currently have structured outputs support, but we can use the existing structured outputs functionality from `gpt-4o`/`gpt-4o-mini` by chaining two requests together. 

This currently requires two calls, but it can allow your functionality to be type-safe and align with how you use the structured outputs already. 